In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [3]:
train = pd.read_csv('restaurant-revenue-prediction/train.csv.zip')
x_test = pd.read_csv('restaurant-revenue-prediction/test.csv.zip')
print(train.shape)
print(x_test.shape)

(137, 43)
(100000, 42)


In [4]:
train.shape

(137, 43)

In [5]:
x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
print(x_train,"\n\n")
# print(y_train)

      Id   Open Date        City  City Group Type  P1   P2   P3   P4  P5  ...  \
0      0  07/17/1999    İstanbul  Big Cities   IL   4  5.0  4.0  4.0   2  ...   
1      1  02/14/2008      Ankara  Big Cities   FC   4  5.0  4.0  4.0   1  ...   
2      2  03/09/2013  Diyarbakır       Other   IL   2  4.0  2.0  5.0   2  ...   
3      3  02/02/2012       Tokat       Other   IL   6  4.5  6.0  6.0   4  ...   
4      4  05/09/2009   Gaziantep       Other   IL   3  4.0  3.0  4.0   2  ...   
..   ...         ...         ...         ...  ...  ..  ...  ...  ...  ..  ...   
132  132  06/25/2008     Trabzon       Other   FC   2  3.0  3.0  5.0   4  ...   
133  133  10/12/2006       İzmir  Big Cities   FC   4  5.0  4.0  4.0   2  ...   
134  134  07/08/2006     Kayseri       Other   FC   3  4.0  4.0  4.0   2  ...   
135  135  10/29/2010    İstanbul  Big Cities   FC   4  5.0  4.0  5.0   2  ...   
136  136  09/01/2009    İstanbul  Big Cities   FC   4  5.0  3.0  5.0   2  ...   

     P28  P29  P30  P31  P3

In [6]:
columns = ['Open Date','City','City Group','Type']
x_train[columns] = x_train[columns].apply(preprocessing.LabelEncoder().fit_transform)
x_train = x_train.drop(['Id'],axis=1)
x_test = x_test.drop(['Id'],axis=1)
x_test[columns] = x_test[columns].apply(preprocessing.LabelEncoder().fit_transform)
print(x_train.shape)
print(y_train.shape)
# print(x_train)

(137, 41)
(137,)


In [7]:
sscaler = StandardScaler()
test_scaler = StandardScaler()

x_train = sscaler.fit_transform(x_train)
y_new = pd.DataFrame(y_train)
y_train = sscaler.fit_transform(y_new)


x_test = pd.DataFrame(x_test)
x_test = test_scaler.fit_transform(x_test)
print(x_test.shape)

(100000, 41)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train,y_train,test_size=0.2,random_state=42)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((109, 41), (109, 1), (28, 41), (28, 1))

In [9]:
model = Sequential()
model.add(Dense(100,input_dim=41,activation='tanh'))
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(1))

In [10]:
model.compile(loss='MSE',optimizer=Adam(learning_rate=0.01),metrics=['RootMeanSquaredError'])

In [12]:
model.fit(X_train,Y_train,epochs=500,validation_data=(X_test,Y_test))

Epoch 1/500
4/4 [==============================] - 0s 10ms/step - loss: 0.1476 - root_mean_squared_error: 0.3842 - val_loss: 1.4791 - val_root_mean_squared_error: 1.2162
Epoch 2/500
4/4 [==============================] - 0s 14ms/step - loss: 0.1346 - root_mean_squared_error: 0.3669 - val_loss: 1.4617 - val_root_mean_squared_error: 1.2090
Epoch 3/500
4/4 [==============================] - 0s 13ms/step - loss: 0.0997 - root_mean_squared_error: 0.3158 - val_loss: 1.5718 - val_root_mean_squared_error: 1.2537
Epoch 4/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0722 - root_mean_squared_error: 0.2687 - val_loss: 1.4326 - val_root_mean_squared_error: 1.1969
Epoch 5/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0778 - root_mean_squared_error: 0.2789 - val_loss: 1.7510 - val_root_mean_squared_error: 1.3233
Epoch 6/500
4/4 [==============================] - 0s 4ms/step - loss: 0.0770 - root_mean_squared_error: 0.2775 - val_loss: 1.6965 - val_root_mean_squar

In [13]:
result = model.predict(x_test)
result.shape

3125/3125 [==============================] - 1s 237us/step


(100000, 1)

In [16]:
predicted = sscaler.inverse_transform(result)
predicted = pd.DataFrame(predicted)
predicted

,0
0,4702675.50
1,2332591.75
2,3334286.00
3,3625818.75
4,4862686.00
...,...
99995,2822179.25
99996,2577731.00
99997,7656332.00
99998,6427792.00


In [19]:
model.evaluate(x_test)

3125/3125 [==============================] - 1s 229us/step - loss: 0.0000e+00 - root_mean_squared_error: 0.0000e+00


[0.0, 0.0]

In [20]:
pred_file = predicted.to_csv('submission.csv')